In [1]:
%matplotlib inline

import sys
import numpy as np
import tensorflow as tf
from matplotlib import pyplot as plt

import aflw_dataset
import alexnet_model
import alexnet_util

reload(aflw_dataset);
reload(alexnet_model);
reload(alexnet_util);

In [2]:
face_db = aflw_dataset.FaceDb("../aflw/aflw/data/aflw.sqlite",
                              "../aflw/aflw/data/flickr/")

In [3]:
train_size = int(len(face_db.face_data) * 0.9)
test_size = len(face_db.face_data) - train_size
print "train_size = %d"%train_size
print "test_size = %d"%test_size

train_size = 19008
test_size = 2113


In [4]:
train_face_dataset = aflw_dataset.Dataset(face_db, [0, train_size])
test_face_dataset = aflw_dataset.Dataset(face_db, [train_size, train_size + test_size])

In [5]:
bvlc_init_data = np.load("bvlc_alexnet.npy").item()

In [6]:
class Model(object):
    
    def __init__(self, bvlc_init_data):
        self._m = alexnet_model.EzModel(
            num_outputs=1, initial_input_biases=[108, 91, 82])

        optimizer = tf.train.AdamOptimizer(learning_rate=0.0001)
        self.loss = self._m.loss
        self.train_op = optimizer.minimize(self.loss)
        self.prec = tf.reduce_mean(tf.cast(
            tf.equal(tf.cast(self._m.probs >= 0.5, tf.int32),
                     tf.cast(self._m.input_labels, tf.int32)), tf.float32))
        
        self.saver = tf.train.Saver(tf.all_variables())
        
        self.init_ops = alexnet_util.make_bvlc_init_ops(
            self._m, bvlc_init_data, fc_ids=[6, 7])

def next_batch(face_dataset):
    images, labels = train_face_dataset.next_batch(
        alexnet_model.EzModel.image_width,
        alexnet_model.EzModel.image_height)
    labels = np.reshape(np.asarray(labels), [-1, 1])
    return images, labels

In [7]:
test_images, test_labels = next_batch(test_face_dataset)

In [ ]:
with tf.Graph().as_default(), tf.Session() as sess:
    with tf.device("/cpu:0"):
        m = Model(bvlc_init_data)
    
    sess.run(tf.initialize_all_variables())
    sess.run(m.init_ops)
    
    for i in range(100000):
        train_images, train_labels = next_batch(train_face_dataset)
        _, train_loss, train_prec= sess.run(
            [m.train_op, m.loss, m.prec],
            feed_dict={m._m.input_images: train_images, m._m.input_labels: train_labels})
        if i % 10 == 0:
            m.saver.save(sess, "face_train_log/face.ckpt", global_step=i)
            test_loss, test_prec= sess.run(
                [m.loss, m.prec],
                feed_dict={m._m.input_images: test_images, m._m.input_labels: test_labels})
            print "%s step %d"%("=" * 10, i)
            print "epoch = %d, progress = %.2f%%"%(
                train_face_dataset.epoch, train_face_dataset.progress)
            print "train_loss = %.6f, train_prec = %.6f"%(train_loss, train_prec)
            print "test_loss = %.6f, test_prec = %.6f"%(test_loss, test_prec)
            sys.stdout.flush()

conv1.shape = (?, 57, 57, 96)
lrn1.shape = (?, 57, 57, 96)
maxpool1.shape = (?, 28, 28, 96)
conv2.shape = (?, 28, 28, 256)
lrn2.shape = (?, 28, 28, 256)
maxpool2.shape = (?, 13, 13, 256)
conv3.shape = (?, 13, 13, 384)
conv4.shape = (?, 13, 13, 384)
conv5.shape = (?, 13, 13, 256)
maxpool5.shape = (?, 6, 6, 256)
fc6.shape = (?, 1, 1, 4096)
fc7.shape = (?, 1, 1, 4096)
fc8.shape = (?, 1, 1, 1)
probs.shape = (?, 1, 1, 1)
========== step 0
epoch = 0, progress = 0.01%
train_loss = 0.685719, train_prec = 0.578125
test_loss = 0.754152, test_prec = 0.765625
========== step 10
epoch = 0, progress = 0.03%
train_loss = 0.326021, train_prec = 0.882812
test_loss = 0.317016, test_prec = 0.851562
========== step 20
epoch = 0, progress = 0.04%
train_loss = 0.185326, train_prec = 0.921875
test_loss = 0.186648, test_prec = 0.937500
